In [ ]:
import gradio as gr
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
import matplotlib.dates as mdates


# Define the file path for saving food entries
FOOD_ENTRIES_FILE = 'food_entries.csv'
# Define the file path for saving BMI history
BMI_HISTORY_FILE = 'bmi_history.csv'

def save_food_entries():
    """Saves the current food_entries list to a CSV file."""
    if food_entries:
        df_food = pd.DataFrame(food_entries)
        df_food['date'] = pd.to_datetime(df_food['date']) # Ensure date is datetime
        df_food.to_csv(FOOD_ENTRIES_FILE, index=False)
        print(f"Food entries saved to {FOOD_ENTRIES_FILE}")
    else:
        # If the list is empty, remove the file if it exists
        if os.path.exists(FOOD_ENTRIES_FILE):
            os.remove(FOOD_ENTRIES_FILE)
            print(f"No food entries to save. Removed {FOOD_ENTRIES_FILE}")
        else:
            print("No food entries to save.")


def load_food_entries():
    """Loads food entries from a CSV file if it exists."""
    global food_entries
    if os.path.exists(FOOD_ENTRIES_FILE):
        try:
            df_food = pd.read_csv(FOOD_ENTRIES_FILE)
            # Convert date column to datetime objects
            df_food['date'] = pd.to_datetime(df_food['date'])
            # Convert DataFrame back to a list of dictionaries
            food_entries = df_food.to_dict('records')
            print(f"Loaded {len(food_entries)} food entries from {FOOD_ENTRIES_FILE}")
        except pd.errors.EmptyDataError:
            food_entries = []
            print(f"'{FOOD_ENTRIES_FILE}' is empty. Initializing with an empty list.")
        except Exception as e:
            food_entries = []
            print(f"Error loading food entries from {FOOD_ENTRIES_FILE}: {e}. Initializing with an empty list.")
    else:
        food_entries = []
        print(f"'{FOOD_ENTRIES_FILE}' not found. Initializing with an empty list.")

# Load food entries when the script starts
load_food_entries()

# Ensure TDEE is defined if it doesn't exist (using a default or placeholder if necessary)
# Using the calculated TDEE from the previous cell's output
 # Default or placeholder TDEE if not calculated in previous tabs
if 'TDEE' not in globals():
    TDEE = 2000.0 # Initialize TDEE as a float

def add_food_entry(food_item: str, calories: float, date=None):
    """Adds a food entry with its calorie count and the provided date or current date."""
    entry_date = date if date else datetime.now()
    try:
        calories = float(calories) # Ensure calories is a float
        food_entry = {
            'food_item': food_item,
            'calories': calories,
            'date': entry_date
        }
        food_entries.append(food_entry)
        save_food_entries() # Save after adding an entry
        return "Food entry added successfully!"
    except ValueError:
        return "Invalid calorie value. Please enter a number."


def calculate_daily_total(date=None):
    """Calculates the total calories consumed on a given date or today's date."""
    daily_total = 0
    target_date = date if date else datetime.now().date()

    for entry in food_entries:
        if entry['date'].date() == target_date:
            daily_total += entry['calories']
    return daily_total


def plot_recent_week_intake():
    """Generates a stacked bar chart of daily calorie intake over the last week."""
    if not food_entries:
        return None # Return None if no food entries to plot

    df_food = pd.DataFrame(food_entries)
    df_food['date'] = pd.to_datetime(df_food['date'])

    # Group by date and food item to get calories per food item per day
    daily_food_intake = df_food.groupby([df_food['date'].dt.date, 'food_item'])['calories'].sum().unstack(fill_value=0)
    daily_food_intake.index = pd.to_datetime(daily_food_intake.index)

    # Filter the grouped data to include only the last 7 days
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=6)
    recent_week_intake = daily_food_intake[(daily_food_intake.index.date >= start_date) & (daily_food_intake.index.date <= end_date)]

    if recent_week_intake.empty:
        return None # Return None if no data for the last week

    # Create a stacked bar plot
    ax = recent_week_intake.plot(kind='bar', stacked=True, figsize=(10, 6))

    plt.xlabel("Date")
    plt.ylabel("Calories Consumed")
    plt.title("Daily Calorie Intake Over the Last Week (Stacked)")
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y')
    plt.tight_layout()

    # Save the plot to a temporary file
    plot_path = "recent_week_intake_stacked.png"
    plt.savefig(plot_path)
    plt.close() # Close the plot to free memory

    return plot_path


def list_food_entries():
    """Lists all recorded food entries with their index, food item, calories, and date."""
    if not food_entries:
        return "No food entries recorded yet."
    entry_list = "Food Entries:\n"
    for i, entry in enumerate(food_entries):
        entry_list += f"{i}: {entry['food_item']} - {entry['calories']:.2f} calories on {entry['date'].strftime('%Y-%m-%d')}\n"
    return entry_list

def remove_food_entry(index_to_remove: int):
    """Removes a food entry based on its index in the list."""
    try:
        index = int(index_to_remove)
        if 0 <= index < len(food_entries):
            removed_entry = food_entries.pop(index)
            save_food_entries() # Save after removing an entry
            return f"Removed entry: {removed_entry['food_item']} - {removed_entry['calories']:.2f} calories."
        else:
            return "Invalid index. Please enter a valid index from the list."
    except ValueError:
        return "Invalid input. Please enter the index of the entry to remove."


def food_tracker_interface(food_item, calories, index_to_remove, tdee_value):
    """Gradio interface function for the Food Tracker tab."""
    # Only add food entry if food_item and calories are provided
    add_message = ""
    if food_item is not None and calories is not None:
        add_message = add_food_entry(food_item, calories)

    remove_message = remove_food_entry(index_to_remove) if index_to_remove is not None else ""

    daily_total = calculate_daily_total()

    # Use the TDEE value passed from the BMR/TDEE tab
    current_tdee = tdee_value if tdee_value is not None else TDEE # Use passed value if available, otherwise use the global default

    calorie_difference = daily_total - current_tdee
    difference_message = f"Today's calorie intake: {daily_total:.2f}\nYour estimated TDEE: {current_tdee:.2f}\n"
    if calorie_difference > 0:
        difference_message += f"You are {abs(calorie_difference):.2f} calories OVER your TDEE for today."
    elif calorie_difference < 0:
        difference_message += f"You are {abs(calorie_difference):.2f} calories UNDER your TDEE for today."
    else:
        difference_message += "You are exactly at your TDEE for today."

    plot_path = plot_recent_week_intake()
    food_list_text = list_food_entries()

    return f"Daily Total Calories: {daily_total:.2f}", difference_message, plot_path, food_list_text, remove_message


# Initialize bmi_history list and load from file
bmi_history = []
def save_bmi_history():
    """Saves the current bmi_history list to a CSV file."""
    if bmi_history:
        df_bmi = pd.DataFrame(bmi_history)
        df_bmi['date'] = pd.to_datetime(df_bmi['date']) # Ensure date is datetime
        df_bmi.to_csv(BMI_HISTORY_FILE, index=False)
        print(f"BMI history saved to {BMI_HISTORY_FILE}")
    else:
        # If the list is empty, remove the file if it exists
        if os.path.exists(BMI_HISTORY_FILE):
            os.remove(BMI_HISTORY_FILE)
            print(f"No BMI history to save. Removed {BMI_HISTORY_FILE}")
        else:
            print("No BMI history to save.")


def load_bmi_history():
    """Loads BMI history from a CSV file if it exists."""
    global bmi_history
    if os.path.exists(BMI_HISTORY_FILE):
        try:
            df_bmi = pd.read_csv(BMI_HISTORY_FILE)
            # Convert date column to datetime objects
            df_bmi['date'] = pd.to_datetime(df_bmi['date'])
            # Convert DataFrame back to a list of dictionaries
            bmi_history = df_bmi.to_dict('records')
            print(f"Loaded {len(bmi_history)} BMI entries from {BMI_HISTORY_FILE}")
        except pd.errors.EmptyDataError:
            bmi_history = []
            print(f"'{BMI_HISTORY_FILE}' is empty. Initializing with an empty list.")
        except Exception as e:
            bmi_history = []
            print(f"Error loading BMI history from {BMI_HISTORY_FILE}: {e}. Initializing with an empty list.")
    else:
        bmi_history = []
        print(f"'{BMI_HISTORY_FILE}' not found. Initializing with an empty list.")

# Load BMI history when the script starts
load_bmi_history()

def plot_bmi_history():
    """Generates a line plot of BMI over time, showing each day on the x-axis."""
    if not bmi_history:
        return None # Return None if no BMI history to plot

    df_bmi = pd.DataFrame(bmi_history)
    df_bmi['date'] = pd.to_datetime(df_bmi['date'])

    # Sort by date to ensure correct plotting order
    df_bmi = df_bmi.sort_values(by='date')

    plt.figure(figsize=(10, 6))
    plt.plot(df_bmi['date'], df_bmi['bmi'], marker='o')
    plt.xlabel("Date")
    plt.ylabel("BMI")
    plt.title("BMI Over Time")
    plt.grid(True)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%Y')) # Format date as DD/MM/YYYY
    plt.gca().xaxis.set_major_locator(mdates.DayLocator()) # Set locator to show each day
    plt.gcf().autofmt_xdate() # Auto-format date labels on x-axis

    # Save the plot to a temporary file
    plot_path = "bmi_history_plot.png"
    plt.savefig(plot_path)
    plt.close() # Close the plot to free memory

    return plot_path


def bmi_calculator_interface(name, weight, height):
    """Gradio interface function for BMI Calculator tab."""
    try:
        weight = float(weight)
        height = float(height) / 100 # Assuming height is in cm
        BMI = (weight) / (height * height)
        if BMI < 18.5:
            category = "Underweight"
        elif BMI >= 18.5 and BMI < 25:
            category = "Normal weight"
        elif BMI >= 25 and BMI < 30:
            category = "Overweight"
        else:
            category = "Obese"

        # Add current BMI to history and save
        bmi_history.append({'date': datetime.now(), 'bmi': BMI})
        save_bmi_history()

        plot_path = plot_bmi_history()

        return f"Hello {name}, your BMI is {BMI:.2f}. Category: {category}", plot_path
    except ValueError:
        return "Please enter valid numerical values for weight and height.", None


def bmr_tdee_calculator_interface(weight, height, age, gender, activity_level):
    """Placeholder for BMR/TDEE Calculator tab."""
    global TDEE # Declare TDEE as global to modify the global variable
    try:
        weight = float(weight)
        height = float(height)
        age = float(age)

        if gender.lower() == "male":
            BMR = 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
        elif gender.lower() == "female":
            BMR = 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)
        else:
            return "Invalid gender entered.", None # Return error message and None for TDEE

        activity_factors = {
            "sedentary": 1.2,
            "lightly active": 1.375,
            "moderately active": 1.55,
            "very active": 1.725,
            "extra active": 1.9
        }

        activity_level_lower = activity_level.lower()

        if activity_level_lower in activity_factors:
            TDEE_calc = BMR * activity_factors[activity_level_lower] # Assign to the global TDEE
            calories_for_weight_loss = TDEE_calc - 500
            calories_for_maintenance = TDEE_calc
            calories_for_weight_gain = TDEE_calc + 500
            output_text = (f"Your Basal Metabolic Rate (BMR) is: {BMR:.2f}\n"
                           f"Your Total Daily Energy Expenditure (TDEE) is: {TDEE_calc:.2f}\n"
                           f"Calorie intake for weight loss: {calories_for_weight_loss:.2f}\n"
                           f"Calorie intake for maintenance: {calories_for_maintenance:.2f}\n"
                           f"Calorie intake for weight gain: {calories_for_weight_gain:.2f}")
            return output_text, TDEE_calc # Return output text and calculated TDEE
        else:
            return "Invalid activity level entered.", None # Return error message and None for TDEE

    except ValueError:
        return "Please enter valid numerical values for weight, height, and age.", None # Return error message and None for TDEE


# Create the Gradio interfaces for each tab
with gr.Blocks() as bmi_tab_content:
    gr.Label("BMI Calculator")
    name_input_bmi = gr.Textbox(label="Name")
    weight_input_bmi = gr.Number(label="Weight in kg")
    height_input_bmi = gr.Number(label="Height in cm")
    calculate_bmi_button = gr.Button("Calculate BMI")
    bmi_output = gr.Textbox(label="BMI Calculation")
    bmi_plot_output = gr.Image(label="BMI Over Time Plot", type="filepath")

    calculate_bmi_button.click(
        fn=bmi_calculator_interface,
        inputs=[name_input_bmi, weight_input_bmi, height_input_bmi],
        outputs=[bmi_output, bmi_plot_output]
    )

with gr.Blocks() as bmr_tdee_tab_content:
    gr.Label("BMR/TDEE Calculator")
    weight_input_bmr = gr.Number(label="Weight in kg")
    height_input_bmr = gr.Number(label="Height in cm")
    age_input_bmr = gr.Number(label="Age in years")
    gender_input_bmr = gr.Radio(["male", "female"], label="Gender")
    activity_level_input_bmr = gr.Dropdown(["sedentary", "lightly active", "moderately active", "very active", "extra active"], label="Activity Level")
    calculate_bmr_button = gr.Button("Calculate BMR/TDEE")
    bmr_tdee_output = gr.Textbox(label="BMR/TDEE Calculation")
    calculated_tdee_state = gr.State(None) # State variable to hold calculated TDEE

    calculate_bmr_button.click(
        fn=bmr_tdee_calculator_interface,
        inputs=[weight_input_bmr, height_input_bmr, age_input_bmr, gender_input_bmr, activity_level_input_bmr],
        outputs=[bmr_tdee_output, calculated_tdee_state]
    )

with gr.Blocks() as food_tracker_tab_content:
    gr.Label("Food Tracker")
    with gr.Row():
        food_item_input = gr.Textbox(label="Food Item")
        calories_input = gr.Number(label="Calories")
    with gr.Row():
        add_button = gr.Button("Add Food Entry")
    with gr.Row():
        index_to_remove_input = gr.Number(label="Index of entry to remove")
        remove_button = gr.Button("Remove Food Entry")

    daily_total_output = gr.Textbox(label="Daily Total Calories")
    difference_message_output = gr.Textbox(label="Calorie Comparison to TDEE")
    food_list_output = gr.Textbox(label="Food Entries")
    remove_message_output = gr.Textbox(label="Remove Status")
    plot_output = gr.Image(label="Recent Week Calorie Intake Plot", type="filepath")


    add_button.click(
        fn=food_tracker_interface,
        inputs=[food_item_input, calories_input, gr.State(None), calculated_tdee_state], # Pass calculated_tdee_state
        outputs=[daily_total_output, difference_message_output, plot_output, food_list_output, remove_message_output]
    )

    remove_button.click(
        fn=food_tracker_interface,
        inputs=[gr.State(None), gr.State(None), index_to_remove_input, calculated_tdee_state], # Pass calculated_tdee_state
        outputs=[daily_total_output, difference_message_output, plot_output, food_list_output, remove_message_output]
    )

# Combine interfaces into a single Gradio app with tabs
demo = gr.TabbedInterface([bmi_tab_content, bmr_tdee_tab_content, food_tracker_tab_content], ["👨🏻‍⚕️ BMI Calculator", "🏋🏻‍♂️ BMR/TDEE Calculator", "🥪 Food Tracker"], theme='soft')

# Launch the Gradio app
demo.launch()

Loaded 2 food entries from food_entries.csv
Loaded 4 BMI entries from bmi_history.csv
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c608c7a88a9eba1759.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
